In [ ]:
pwd

In [153]:
# using Pandas from my virtualenv
import os
import sys
sys.path.append( os.environ['VIRTUAL_ENV'] + "/lib/python3.9/site-packages" )
sys.path

['/Users/mvferr/work/dev/fund',
 '/usr/local/Cellar/python@3.9/3.9.1_1/Frameworks/Python.framework/Versions/3.9/lib/python39.zip',
 '/usr/local/Cellar/python@3.9/3.9.1_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9',
 '/usr/local/Cellar/python@3.9/3.9.1_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/lib-dynload',
 '',
 '/usr/local/Cellar/jupyterlab/2.2.9_1/libexec/lib/python3.9/site-packages',
 '/usr/local/Cellar/jupyterlab/2.2.9_1/libexec/lib/python3.9/site-packages/IPython/extensions',
 '/Users/mvferr/.ipython',
 '../venv/lib/python3.9/site-packages',
 '/Users/mvferr/work/dev/fund/venv/lib/python3.9/site-packages',
 '/Users/mvferr/work/dev/fund/venv/lib/python3.9/site-packages']

In [ ]:
import requests

url = 'http://www.fundamentus.com.br/resultado.php'
hdr = {'User-agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
       'Accept': 'text/html, text/plain, text/css, text/sgml, */*;q=0.01',
       'Accept-Encoding': 'gzip, deflate',
       }
params = {'pl_min': '0.01'}

content = requests.post(url, headers=hdr, data=params)

In [ ]:
import pandas as pd

## parse
df = pd.read_html(content.text, decimal=",", thousands='.')[0]

In [ ]:
def fix_perc(df, column):
    """
    Fix percent:
      - inplace: replace string in pt-br
      - from '45,56%' to '0.4556'

    Input: DataFrame, column_name
    """

    df[column] = df[column].str.rstrip('%')
    df[column] = df[column].str.replace('.', '' )
    df[column] = df[column].str.replace(',', '.')
    df[column] = df[column].astype(float) / 100     
    

## Fix: percent string
fix_perc(df,'Div.Yield'    )
fix_perc(df,'Mrg Ebit'     )
fix_perc(df,'Mrg. Líq.'    )
fix_perc(df,'ROIC'         )
fix_perc(df,'ROE'          )
fix_perc(df,'Cresc. Rec.5a')



In [ ]:
from tabulate import tabulate
# tablefmt: plain,simple,presto
x= tabulate(df[ ['Papel','Cotação', 'P/L', 'ROIC','ROE','Liq.2meses','Patrim. Líq'] ]
                , tablefmt='presto'
                , showindex='no'
                , headers=['Papel','Cotação', 'P/L', 'ROIC','ROE','Liq.2meses','Patrim. Líq']
                , floatfmt=".2f"
                , colalign=["left","right","right","right","right"]
                , disable_numparse=False
               )

print(df.to_csv(index=False, header=True, decimal='.', float_format='%.2f' ))

In [ ]:
df.name = 'Fundamentus'

# update index
df.index = df['Papel']
df.index.name = 'Papel'

# rename columns
df3 = pd.DataFrame()
df3['papel'] = df['Papel']
df3['pl']    = df['P/L']
df3['pvp']   = df['P/VP']
df3['patrliq'] = df['Patrim. Líq']


df3
print(df3.to_csv(index=False, header=True, sep=';', doublequote='"', decimal='.', float_format='%1.2f' ))

In [ ]:
df3

In [ ]:
df

df[ df['P/L'] > 0 ]
df[ df['P/L'] > 0 &   df['ROE'] > 0 ]
df[ df['P/L'] > 0 &&  df['ROE'] > 0 ]
df[ df['P/L'] > 0 and df['ROE'] > 0 ]

df[ df['P/L'] > 0  ][ ['Papel','EV/EBITDA','ROE','ROIC'] ]

df.name = 'Fundamentus'
df.index.name = 'Papel'

# rename index
df.index = df['Papel']


df2 = pd.DataFrame( df               , columns=['Cotação', 'P/L', 'ROE', 'ROIC', 'EV/EBIT','EV/EBITDA'] )
df2 = pd.DataFrame( df[ df['P/L']>0 ], columns=['Cotação', 'P/L', 'ROE', 'ROIC', 'EV/EBIT','EV/EBITDA'] )
